In [37]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

# Set style and settings
plt.style.use('ggplot')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 15)

In [33]:
# Load data and set Datetime column
collisions = pd.read_csv('../data/external/Collisions.csv', 
                         parse_dates={'Datetime': ['INCDTTM']}, 
                         infer_datetime_format=True)

In [34]:
# Clean up and set index to datetime
collisions = (
    collisions.set_index('Datetime')
    .sort_index()
    .drop(columns=['INCDATE', 'EXCEPTRSNDESC'])
)

In [35]:
collisions.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 220436 entries, 2003-10-06 00:00:00 to 2020-07-14 00:00:00
Data columns (total 38 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   X                212975 non-null  float64
 1   Y                212975 non-null  float64
 2   OBJECTID         220436 non-null  int64  
 3   INCKEY           220436 non-null  int64  
 4   COLDETKEY        220436 non-null  int64  
 5   REPORTNO         220436 non-null  object 
 6   STATUS           220436 non-null  object 
 7   ADDRTYPE         216729 non-null  object 
 8   INTKEY           71516 non-null   float64
 9   LOCATION         215860 non-null  object 
 10  EXCEPTRSNCODE    100033 non-null  object 
 11  EXCEPTRSNDESC    11746 non-null   object 
 12  SEVERITYCODE     220435 non-null  object 
 13  SEVERITYDESC     220436 non-null  object 
 14  COLLISIONTYPE    194284 non-null  object 
 15  PERSONCOUNT      220436 non-null  int64  
 16  PEDC

In [36]:
collisions.describe()

,X,Y,OBJECTID,INCKEY,COLDETKEY,INTKEY,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,SDOT_COLCODE,SDOTCOLNUM,SEGLANEKEY,CROSSWALKKEY
count,212975.000000,212975.000000,220436.000000,220436.000000,220436.000000,71516.000000,220436.000000,220436.000000,220436.000000,220436.000000,220436.000000,220436.000000,220436.000000,220435.000000,1.272050e+05,220436.000000,2.204360e+05
mean,-122.330748,47.620232,110218.500000,143893.625937,144116.360395,37614.720790,2.227145,0.038156,0.027310,1.730697,0.373868,0.015165,0.001674,13.382362,7.971063e+06,262.425969,9.617439e+03
std,0.030042,0.056009,63634.536307,88451.013280,88818.314232,51915.702891,1.471406,0.201881,0.164398,0.829041,0.732352,0.158077,0.044493,7.305444,2.611523e+06,3254.921395,7.160964e+04
min,-122.419091,47.495573,1.000000,1001.000000,1001.000000,23807.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.007024e+06,0.000000,0.000000e+00
25%,-122.349280,47.577331,55109.750000,71395.750000,71395.750000,28652.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,11.000000,6.007029e+06,0.000000,0.000000e+00
50%,-122.330348,47.616106,110218.500000,126707.500000,126707.500000,29973.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,11.000000,8.033011e+06,0.000000,0.000000e+00
75%,-122.311998,47.664261,165327.250000,208526.250000,208726.250000,33983.000000,3.000000,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,14.000000,1.018101e+07,0.000000,0.000000e+00
max,-122.238949,47.734142,220436.000000,332640.000000,334140.000000,757580.000000,93.000000,6.000000,2.000000,15.000000,78.000000,41.000000,5.000000,87.000000,1.307202e+07,525241.000000,5.239700e+06


In [40]:
# Pandas profiling, use minimal=True for large dataset
profile = ProfileReport(collisions, title='Collisions Profile Report', minimal=True)
profile.to_file('../reports/collisions_profiling.html')
